In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [4]:
dataset = pd.read_csv('house.csv')
dataset.drop(['Id'],
             axis=1,
             inplace=True)

dataset['SalePrice'] = dataset['SalePrice'].fillna(
  dataset['SalePrice'].mean())


new_dataset = dataset.dropna()
s = (new_dataset.dtypes == 'object')
object_cols = list(s[s].index)
OH_encoder = OneHotEncoder(sparse_output=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(new_dataset[object_cols]))
OH_cols.index = new_dataset.index
OH_cols.columns = OH_encoder.get_feature_names_out()
df_final = new_dataset.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

X = df_final.drop('SalePrice', axis=1)  # Specify the column name for the target variable
y = df_final['SalePrice']  # Specify the column name for the target variable

# Split the data into train and test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)




In [45]:
train_y

2356    180921.19589
1789    180921.19589
2371    180921.19589
930     201000.00000
2816    180921.19589
            ...     
1638    180921.19589
1095    176432.00000
1130    135000.00000
1294    115000.00000
860     189950.00000
Name: SalePrice, Length: 2330, dtype: float64

In [6]:
def relu(x):
    return np.where(x > 0, x, 0)
def drelu(x):
    return np.where(x > 0, 1,0)
def mse(t,p):
    """
    Returns mean sqaured error given two arrays of the same length
    """
    return sum((t-p)**2)/len(t)

def softmax(x):
    # assumes x is a vector

    return np.exp(x) / np.sum(np.exp(x))

In [74]:
class NeuralNetwork:
    def __init__(self,hidden = []) -> None:
        self.sizes = hidden
        self.biases = None
        self.weights = None
        self.type = None
        self.map = {}
    # detects what type of model we want and constructs our bias,weights,type and map dependent on the dataframe and series given
    def construct(self,X,Y):
        inputsize = X.shape[1]
        self.sizes.insert(0,inputsize)
        # Checking if its categorical or regression NN
        if Y.dtype == 'float64':
            self.sizes.append(1)
            self.type = 'regression'
        else:
            unique_values = len(Y.unique())
            self.sizes.append(unique_values)
            self.type = 'classification'
            self.map = {index: value for index, value in enumerate(Y.unique())}
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(self.sizes[:-1],self.sizes[1:])]

    # Helper for train function
    # Feeds our input into our neural network and determines the loss 
    def forward(self,input,answers):
        ## Input is supposed to be the training matrix
        input = input.T
        for b, w in zip(self.biases, self.weights):
            dot = np.dot(w, input)+b
            if np.array_equal(self.biases[len(self.biases) - 1] ,b):
                if self.type == 'classification':
                    input = softmax(dot.T)
            else:
                input = relu(dot)
        # input is now the output activation
        # If the type is regression, then the length of the output activation should be 1. 
        if self.type == 'classification':
            ## Accuracy instead of loss 
            currentsum = 0
            for row,index in zip(input,answers):
                if np.argmax(row) == index:
                    currentsum += 1
            loss = currentsum
        return  loss
    
    # Returns a tuple of dw,db, which are layer by layer arrays that represent partials
    def backprop(self,inputx,inputy):
        # input x is a 1d vector ,input y is a scalar
        partialb = [np.zeros(b.shape) for b in self.biases]
        partialw = [np.zeros(w.shape) for w in self.weights]
        storedsums = []
        input = inputx.reshape(inputx.size,1)
        storedactivations = [input.T]
        # Forward propagation, storing the su
        for b, w in zip(self.biases, self.weights):
            dot = np.dot(w, input)+b
            storedsums.append(dot)
            ## If we are on the last iteration, then our activation function is linear if its regression, or softmax if its classification
            if np.array_equal(self.biases[len(self.biases) - 1] ,b):
                if self.type == 'classification':
                    ## Ensure no overflow
                    input = softmax(dot.T)
            else:
                input = relu(dot)
            storedactivations.append(input.T)
        ## Actual backpropagation
        #First Layer
        ## Regression case. Thus, the activation for our output for our output layer is linear. Our loss function is mean square error, thus delta (dL/dz) is -2(y-z).
        ## Classification case. Delta of the output layer is dependent on the output layer. Credit to https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1 for doing the dirty work
        if self.type == 'classification':
            dz = storedactivations[-1]
            dz[inputy] = dz[inputy] - 1
            assert dz.shape == partialb[-1].shape
            partialb[-1] = dz
            dw = np.dot(dz, storedactivations[-2])
            assert dw.shape == partialw[-1].shape
            partialw[-1]= dw
        
        # Finding partials for the rest of the layers
        for i in range(2,len(self.sizes)): 
            dz = drelu(storedsums[-i])

            delta = np.dot(self.weights[-i + 1].T,partialb[-i + 1]) * dz
            assert delta.shape == partialb[-i].shape
            partialb[-i] = delta   
            weight = np.dot(delta, storedactivations[-i - 1])
            assert weight.shape == partialw[-i].shape
            partialw[-i] = weight
        return (partialw,partialb)

    # Updates our weights and biases with a given batch
    def updatebatch(self,batchx,batchy,learningrate = 0.0001):
        ## Assumptions these are numpy arrays

        currentsumb = [np.zeros(b.shape) for b in self.biases]
        currentsumw = [np.zeros(w.shape) for w in self.weights]
        batchx = batchx.values

        for x,y in zip(batchx,batchy):
            gradientw,gradientb = self.backprop(x,y)
            for i in range(len(currentsumb)):
                currentsumb[i] = (gradientb[i] + currentsumb[i])
                currentsumw[i] = (gradientw[i] + currentsumw[i])
        
        for i in range(len(currentsumb)):
            self.weights[i] = self.weights[i] - (learningrate/len(batchy)) * currentsumw[i]
            self.biases[i] = self.biases[i] - (learningrate/len(batchy)) * currentsumb[i]
    

            
    def train(self,X,Y,batchsize = 32,epoch = 10,testx = None,testy = None):
        assert isinstance(X, pd.DataFrame) 
        assert isinstance(Y,pd.Series)
        # Constructs the array for training and test error
        self.construct(X,Y)
        if testx and testy:
            testloss = [self.forward(testx,testy)]
        trainingloss = [self.forward(X,Y)]
        print(trainingloss)
        if self.type == 'classification':
            Y = pd.factorize(Y)[0]
        minibatchesX = np.array_split(X, len(X) // batchsize)
        minibatchesY = np.array_split(Y, len(Y) // batchsize)
        #split dataframe into batches of size batchsize
        for i in range(epoch):
            for x,y in zip(minibatchesX,minibatchesY):
                self.updatebatch(x,y)
            
            if self.type == 'classification':
                print('Your training accuracy after epoch ' + str(i) + ' is ' + str(self.forward(X,Y)))
                if testx and testy:
                    print('Your test accuracy after epoch ' + str(i) + ' is ' + str(self.forward(testx,testy)))



                


            
            
        


In [75]:
from sklearn.datasets import make_moons, make_blobs

X, y = make_moons(n_samples=100, noise=0.1)

y = y*2 - 1 # make y be -1 or1
trainx = pd.DataFrame(X)
trainy = pd.Series(y)

In [76]:
train_y

2356    180921.19589
1789    180921.19589
2371    180921.19589
930     201000.00000
2816    180921.19589
            ...     
1638    180921.19589
1095    176432.00000
1130    135000.00000
1294    115000.00000
860     189950.00000
Name: SalePrice, Length: 2330, dtype: float64

In [77]:
nn = NeuralNetwork([30,20,30] )
nn.train(train_X,train_y)



[138667173677.29596]
180921.19589041095 [[-7945.9507598]]
180921.19589041095 [[-178777.09432143]]
180921.19589041095 [[-13731.39855112]]
201000.0 [[-51389.97162079]]
180921.19589041095 [[-213923.16317249]]
180921.19589041095 [[-35985.27527704]]
153900.0 [[-214973.64330788]]
180921.19589041095 [[-66021.09861474]]
180921.19589041095 [[-272078.62796439]]
212000.0 [[-157323.91725531]]
180921.19589041095 [[-150214.7023273]]
194500.0 [[-103572.5507115]]
180921.19589041095 [[12162.7395652]]
190000.0 [[-109967.19438225]]
180921.19589041095 [[-93952.49993086]]
103200.0 [[6201.89445041]]
180921.19589041095 [[-105501.49461884]]
173000.0 [[-84278.16904913]]
180921.19589041095 [[-168026.69512678]]
136000.0 [[-97726.67822035]]
180921.19589041095 [[-165388.68073194]]
180921.19589041095 [[48438.95112052]]
206000.0 [[-7623.41263818]]
180921.19589041095 [[-141878.68415531]]
180921.19589041095 [[-102239.34206231]]
174900.0 [[-81753.06818261]]
180921.19589041095 [[-195838.24335076]]
154000.0 [[-142078.897

C:\Users\caryk\AppData\Local\Temp\ipykernel_22420\1339445669.py:99: RuntimeWarning: invalid value encountered in multiply
  delta = np.dot(self.weights[-i + 1].T,partialb[-i + 1]) * dz


AssertionError: 

In [64]:
trainy

0     1
1    -1
2     1
3     1
4    -1
     ..
95   -1
96   -1
97    1
98    1
99    1
Length: 100, dtype: int64